In [0]:
use catalog rodrigo_lakehouse

In [0]:
MERGE INTO rodrigo_lakehouse.gold.dim_channels AS target
USING rodrigo_lakehouse.silver.channels AS source
ON target.channel_id = source.channel_id
WHEN MATCHED THEN
  UPDATE SET 
    target.trade_chnl_desc = source.trade_chnl_desc,
    target.trade_group_desc = source.trade_group_desc,
    target.trade_type_desc = source.trade_type_desc,
    target.start_date = cast('1990-01-01' as timestamp),
    target.end_date = cast('9999-12-31' as timestamp),
    target.current = current,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
  INSERT (channel_id, trade_chnl_desc, trade_group_desc, trade_type_desc, start_date, end_date, current, insertdate)
  VALUES (source.channel_id, source.trade_chnl_desc, source.trade_group_desc, source.trade_type_desc, cast('1990-01-01' as timestamp), cast('9999-12-31' as timestamp), True, source.insertdate)

In [0]:
MERGE WITH SCHEMA EVOLUTION INTO rodrigo_lakehouse.gold.dim_places AS target
USING rodrigo_lakehouse.silver.places AS source
ON target.place_id = source.place_id
WHEN MATCHED THEN
  UPDATE SET 
    target.Btlr_Org_LVL_C_Desc = source.Btlr_Org_LVL_C_Desc,
    target.start_date = cast('1990-01-01' as timestamp),
    target.end_date = cast('9999-12-31' as timestamp),
    target.current = current,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
    INSERT (place_id, Btlr_Org_LVL_C_Desc, start_date, end_date, current, insertdate)
  VALUES (source.place_id, source.Btlr_Org_LVL_C_Desc, cast('1990-01-01' as timestamp), cast('9999-12-31' as timestamp), True, source.insertdate)

In [0]:
MERGE WITH SCHEMA EVOLUTION INTO rodrigo_lakehouse.gold.dim_products AS target
USING rodrigo_lakehouse.silver.products AS source
ON target.product_id = source.product_id
WHEN MATCHED THEN
  UPDATE SET 
    target.ce_brand_flvr = source.ce_brand_flvr,
    target.brand_nm = source.brand_nm,
    target.pkg_cat = source.pkg_cat,
    target.pkg_cat_desc = source.pkg_cat_desc,
    target.tsr_pckg_nm = source.tsr_pckg_nm,
    target.start_date = cast('1990-01-01' as timestamp),
    target.end_date = cast('9999-12-31' as timestamp),
    target.current = current,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
    INSERT (source.product_id, ce_brand_flvr, brand_nm, pkg_cat, pkg_cat_desc, tsr_pckg_nm, start_date, end_date, current, insertdate)
  VALUES (source.product_id, source.ce_brand_flvr, source.brand_nm,  source.pkg_cat,  source.pkg_cat_desc,  source.tsr_pckg_nm, cast('1990-01-01' as timestamp), cast('9999-12-31' as timestamp), True, source.insertdate)

In [0]:
MERGE INTO rodrigo_lakehouse.gold.fact_sales AS target
USING 
(
  SELECT 
    sales.sale_id,
    dim_calendar.skcalendar,
    dim_channels.skchannel,
    dim_products.skproduct,
    dim_places.skplace,
    sales.volume,
    sales.insertdate
  FROM silver.sales
  INNER JOIN gold.dim_calendar using (date)
  INNER JOIN gold.dim_channels using (channel_id)
  INNER JOIN gold.dim_products using (product_id)
  INNER JOIN gold.dim_places using (place_id)
) AS source
ON target.sales_id = source.sale_id
WHEN MATCHED THEN
  UPDATE SET 
    target.sales_id = source.sale_id,
    target.skcalendar = source.skcalendar,
    target.skproduct = source.skproduct,
    target.skplace = source.skplace,
    target.skchannel = source.skchannel,
    target.volume = source.volume,
    target.insertdate = source.insertdate
WHEN NOT MATCHED THEN 
  INSERT (sales_id, skcalendar, skproduct, skplace, skchannel, volume, insertdate)
  VALUES (source.sale_id, source.skcalendar, source.skproduct, source.skplace, source.skchannel, source.volume, source.insertdate)